In [ ]:
notebook vhgfbc01dev_silver → notebooks → RDM मध्ये बनव आणि Silver lakehouse attach कर.

Notebook name: POC_RDM_DemoReference_Add_Detection 

In [ ]:
Cell 0 — Context check (Approved RDM table आहे का?)
%%sql
-- Approved values (SharePoint → Dataflow Gen2 → Silver) इथे असतात
SELECT COUNT(*) AS approved_row_count
FROM silver_rdm_demo_reference;

Cell 1 — Approved values quick view (optional)
%%sql
SELECT
  z_src_system_id,
  care_epi_closure_reason,
  care_epi_closure_reason_conformed
FROM silver_rdm_demo_reference
ORDER BY z_src_system_id, care_epi_closure_reason;

Cell 2 — Demo “source/system” table create (mock transactional source)
%%sql
-- हा table "systems मधून आलेल्या raw values" simulate करतो
DROP TABLE IF EXISTS silver_demo_source_reference;

CREATE TABLE silver_demo_source_reference (
  z_src_system_id     STRING,
  raw_value           STRING,
  z_added_date_time   TIMESTAMP
);

Cell 3 — Source values insert (इथेच तू NEW values टाक)

महत्वाचं: LEFT EARLY आधीच approved मध्ये आहे म्हणालीस, म्हणून इथे “नवीन” value टाकतो.

%%sql
-- Source table reset
DELETE FROM silver_demo_source_reference;

-- NEW raw values (approved मध्ये नसतील अशी)
INSERT INTO silver_demo_source_reference VALUES
('WIP',  'TRANSFERRED OUT', current_timestamp()),
('WIP',  'Admin Closed',    current_timestamp()),
('SONE', 'Moved Away',      current_timestamp()),
('WIP',  'Unknown',         current_timestamp());  -- जर Unknown approved मध्ये असेल तर detect होणार नाही

Cell 4 — Source values verify
%%sql
SELECT *
FROM silver_demo_source_reference
ORDER BY z_src_system_id, raw_value;

Cell 5 — Create _add table (new value detection)
%%sql
-- Output: जे raw values approved table मध्ये नाहीत ते `_add` मध्ये टाकतो
DROP TABLE IF EXISTS silver_rdm_demo_reference_add;

CREATE TABLE silver_rdm_demo_reference_add AS
SELECT
  s.z_src_system_id,
  s.raw_value                          AS care_epi_closure_reason,
  CAST(NULL AS STRING)                 AS care_epi_closure_reason_conformed, -- business later fills
  current_timestamp()                  AS z_record_created_date_time,
  current_user()                       AS z_record_created_by_user,
  current_timestamp()                  AS z_record_modified_date_time,
  current_user()                       AS z_record_modified_by_user
FROM (
  SELECT DISTINCT
    z_src_system_id,
    TRIM(raw_value) AS raw_value
  FROM silver_demo_source_reference
  WHERE raw_value IS NOT NULL AND TRIM(raw_value) <> ''
) s
LEFT ANTI JOIN (
  SELECT DISTINCT
    z_src_system_id,
    TRIM(care_epi_closure_reason) AS care_epi_closure_reason
  FROM silver_rdm_demo_reference
  WHERE care_epi_closure_reason IS NOT NULL AND TRIM(care_epi_closure_reason) <> ''
) a
ON  lower(s.z_src_system_id) = lower(a.z_src_system_id)
AND lower(s.raw_value)       = lower(a.care_epi_closure_reason);

Cell 6 — _add output view (हा Lane B result)
%%sql
SELECT *
FROM silver_rdm_demo_reference_add
ORDER BY z_src_system_id, care_epi_closure_reason;

Cell 7 — Quick sanity: किती “new” values detect झाले?
%%sql
SELECT COUNT(*) AS new_values_detected
FROM silver_rdm_demo_reference_add;


dynaic solution????

In [ ]:
-- Run in a Fabric notebook cell with %sql
DROP TABLE IF EXISTS silver.config_rdm_lists;

CREATE TABLE silver.config_rdm_lists (
    list_name        STRING,
    target_table     STRING,
    keep_columns     STRING,   -- comma-separated list of columns to keep from SharePoint
    rename_map       STRING,   -- semicolon-separated mappings: src=dest;src2=dest2
    data_types_map   STRING,   -- optional: col=type;col2=type2  (ex: z_record_created_date_time=timestamp)
    is_active        INT,
    notes            STRING,
    z_record_created_date_time TIMESTAMP,
    z_record_created_by_user   STRING,
    z_record_modified_date_time TIMESTAMP,
    z_record_modified_by_user   STRING
)
USING DELTA;
